# Object Oriented Idioms

## Identity, Equality, and Aliases

In [5]:
>>> charles = {'name': 'Charles L. Dodgson', 'born': 1832}

>>> lewis = charles # Charles and Lewis are aliases: bounding to the same object

>>> lewis is charles

True

In [11]:
>>> id(charles), id(lewis) # return integer representing its identity, in CPython, return the memory address of the object

# The ID is guaranteed to be a unique numeric lable, it will never change during the life of object

(4450746728, 4450746728)

In [6]:
>>> lewis['balance'] = 950

>>> charles

{'balance': 950, 'born': 1832, 'name': 'Charles L. Dodgson'}

In [7]:
>>> alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

>>> alex == charles # is not alias for charles, bound to different objects, same value

True

In [10]:
>>> alex is charles # but they have different identities

False

- The **==** operator compares the values of objects (the data they hold), while **is** compares their identities.

- We often care about values and not identities, so *==* appears more frequently than *is* in Python code.

- For example, we can use: x is None or x is not None

## Copies are shallow by default

In [13]:
>>> l1 = [3, [55, 44], (7, 8, 9)]

>>> l2 = list(l1) # constructor copy

>>> l2

>>> l2 == l1

True

In [14]:
>>> l2 is l1

False

Using the constructor or [:] produces a shallow copy (i.e., the outermost container is duplicated, but the copy is filled with references to the same items held by the original container). This saves memory and causes no problems if all the items are immutable. But if there are mutable items, this may lead to unpleasant surprises.

### Suggestion: use http://pythontutor.com/visualize.html#mode=edit to check

In [15]:
l1 = [3, [66, 55, 44], (7, 8, 9)] 
l2 = list(l1) 
l1.append(100) 
l1[1].remove(55) 
print('l1:', l1) 

('l1:', [3, [66, 44], (7, 8, 9), 100])


In [16]:
print('l2:', l2) 

('l2:', [3, [66, 44], (7, 8, 9)])


In [18]:
l2[1] += [33, 22]
l2[2] += (10, 11) 
print('l1:', l1)

('l1:', [3, [66, 44, 33, 22, 33, 22], (7, 8, 9), 100])


In [19]:
print('l2:', l2)

('l2:', [3, [66, 44, 33, 22, 33, 22], (7, 8, 9, 10, 11)])


In [28]:
class Bus:
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = [] 
        else:
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)
        
import copy

bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])

bus2 = copy.copy(bus1) # shallow copy

bus3 = copy.deepcopy(bus1) # deep copy, can run into infinite loop

id(bus1), id(bus2), id(bus3)

(4451859128, 4451860280, 4451858768)

In [29]:
>>> bus1.drop('Bill')
>>> bus2.passengers
>>> id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)

(4451858912, 4451858912, 4451858984)

In [30]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

## Function Parameters as Reference

- The only mode of parameter passing in Python is **call by sharing or call by reference**. That is the same mode used in most OO languages, including Ruby, SmallTalk, and Java (this applies to Java reference types; primitive types use call by value). 

- Call by sharing means that each formal parameter of the function gets a **copy of each reference** in the arguments. In other words, the parameters inside the function become *aliases* of the actual arguments.

- The result of this scheme is that a function may change any mutable object passed as a parameter, but it **cannot** change the identity of those objects (i.e., it cannot altogether replace an object with another).

In [31]:
def f(a, b):
    a += b
    return a

x = 1 # immutable object
y = 2
f(x, y)

3

In [32]:
x, y

(1, 2)

In [33]:
a = [1, 2] # mutable object
b = [3, 4]
f(a, b)

[1, 2, 3, 4]

In [34]:
a, b

([1, 2, 3, 4], [3, 4])

In [35]:
t = (10, 20) # immutable object
u = (30, 40)
f(t, u)

(10, 20, 30, 40)

In [36]:
t, u

((10, 20), (30, 40))

### Avoid mutable types as Parameter defaults

In [37]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""
    
    def __init__(self, passengers=[]): # never do that
        self.passengers = passengers
        
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)
        
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

['Alice', 'Bill']

In [38]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

['Bill', 'Charlie']

In [39]:
bus2 = HauntedBus() # Terrible thing happens from here
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [40]:
bus3 = HauntedBus() # Terrible thing happens from here
bus3.passengers

['Carrie']

In [41]:
bus3.pick('Dave')
bus2.passengers

['Carrie', 'Dave']

In [42]:
bus2.passengers is bus3.passengers

True

In [43]:
bus1.passengers

['Bill', 'Charlie']

In [44]:
# Here is how to fix:
class TwilightBus:
    """A bus model that makes passengers vanish"""
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers) # Bus keep its own copy of passengers
            
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

## del and Garbage Collection

The *del* statement deletes *names*, not objects. An object may be garbage collected as result of a *del* command, but **only if** the variable deleted holds the **last reference** to the object, or if the object becomes unreachable. **Rebinding** a variable may also cause the number of references to an object to reach zero, causing its **destruction**.

In [ ]:
>>> import weakref

>>> s1 = {1, 2, 3}

>>> s2 = s1

>>> def bye():

    ... print('Gone with the wind...') ...

>>> ender = weakref.finalize(s1, bye) # weak reference, does not increase its reference count

>>> ender.alive 
True

>>> del s1

>>> ender.alive 
True

>>> s2 = 'spam' 
Gone with the wind...

>>> ender.alive
False